In [1]:
from deepface import DeepFace
import matplotlib.pyplot as plt
import os
import cv2

2024-06-14 16:52:14.801111: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-14 16:52:14.861223: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-14 16:52:15.787659: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
path1 = '/home/liuyichen/deepface/tests/dataset/59555bfc1d694413c3620fda802de70.jpg'
path2 = '/home/liuyichen/deepface/tests/dataset/b5890db4e24a71e73325f6cc853c26c.jpg'

# img1 = DeepFace.detectFace(path1).reshape(224,224,3)
# plt.imshow(img1)

In [ ]:
res = DeepFace.verify(img1_path=path1, img2_path=path2, model_name='VGG-Face')

print(res['verified'])

In [ ]:
video_path = "/home/liuyichen/deepface/video/pigu.mp4"
output_folder = "/home/liuyichen/deepface/temp_frame"

cap = cv2.VideoCapture(video_path)
    
# 检查视频是否打开成功
if not cap.isOpened():
    print("Error: Could not open video.")

else:
    # 获取视频的帧率
    fps = cap.get(cv2.CAP_PROP_FPS)

    # 当前帧计数
    frame_count = 0

    # 确保输出文件夹存在
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # 每秒截取一帧
        if frame_count % int(fps) == 0:
            frame_number = frame_count // int(fps)
            frame_filename = os.path.join(output_folder, f"frame_{frame_number:04d}.jpg")
            cv2.imwrite(frame_filename, frame)
        
        frame_count += 1
        
        dfs = DeepFace.find(
            img_path = frame_filename,
            db_path = "/home/liuyichen/deepface/tests/dataset",
            model_name="VGG-Face"
        )
        if not str(dfs[0]).startswith('Empty'):
            print('Detect True!')
            print(dfs)
            break
        

    # 释放视频捕获对象
    cap.release()
    print("Done.")

In [ ]:
dfs = DeepFace.find(
  img_path = path1,
  db_path = "/home/liuyichen/deepface/tests/dataset",
  model_name="VGG-Face"
)
print(dfs)

In [ ]:
# dict = {}
# pointer = 0
# directory_path = '/home/liuyichen/deepface/tests/dataset'

# def count_all_files(directory_path):
#     file_count = 0

#     # 遍历目录及其子目录
#     for root, dirs, files in os.walk(directory_path):
#         # 累加文件数量
#         file_count += len(files)

#     return file_count

# # 示例
# size_of_database = count_all_files(directory_path)
# print(f"目录 {directory_path} 及其子目录下的文件数量是: {size_of_database}")



In [2]:
dict = {}
pointer = 0
directory_path = '/home/liuyichen/deepface/tests/dataset'

def count_all_files(directory_path):
    file_count = 0

    # 遍历目录及其子目录
    for root, dirs, files in os.walk(directory_path):
        # 累加文件数量
        file_count += len(files)

    return file_count

# 示例
size_of_database = count_all_files(directory_path)
print(f"目录 {directory_path} 及其子目录下的文件数量是: {size_of_database}")

sig_strand = [0] * size_of_database
cat = 0

def is_image_file(file_path):
    # 定义支持的图片文件扩展名
    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.gif'}
    return os.path.splitext(file_path)[1].lower() in image_extensions

def get_file_indices_in_directory(directory_path, file_paths):
    # 获取目录下的所有文件
    all_files = []
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            all_files.append(os.path.join(root, file))
    
    # 查找文件路径在所有文件列表中的索引
    file_indices = [all_files.index(file_path) for file_path in file_paths if file_path in all_files]
    
    return file_indices

def get_file_at_index(directory_path, index):
    # 获取目录下的所有文件
    all_files = []
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            all_files.append(os.path.join(root, file))
    
    # 检查索引是否在范围内
    if index < 0 or index >= len(all_files):
        raise IndexError("Index out of range.")
    
    # 返回第 index 个文件的路径
    return all_files[index]

db_path = "/home/liuyichen/deepface/tests/dataset"
while(pointer < size_of_database):
    print(f'pointer is {pointer}')
    if(sig_strand[pointer] == 0):
        if(is_image_file(get_file_at_index(db_path, pointer))):
            dfs = DeepFace.find(
            img_path = get_file_at_index(db_path, pointer),
            db_path = db_path,
            model_name="VGG-Face"
            )
            # 提取匹配结果中的文件路径
            matched_files = dfs[0]['identity']
            print(f'match file is {matched_files}')

            # 获取文件在目录中的索引
            file_indices = get_file_indices_in_directory(db_path, matched_files)
            sig_strand[pointer] = 1
            # print(f'file_indices is {file_indices}')
            for i in file_indices:
                sig_strand[i] = 1
                
            # file_indices.append(pointer)
            result_list = []
            for i in file_indices:
                result_list.append(get_file_at_index(db_path, i))
                
            dict[cat] = result_list
            cat += 1
        else:
            sig_strand[pointer] = 1
    else:
        pointer += 1
        
for key, value in dict.items():
    print(f"{key}: {value}")
        

目录 /home/liuyichen/deepface/tests/dataset 及其子目录下的文件数量是: 67
pointer is 0
24-06-14 16:52:20 - Searching /home/liuyichen/deepface/tests/dataset/img10.jpg in 70 length datastore


2024-06-14 16:52:24.706522: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2024-06-14 16:52:24.706740: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9410 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:3b:00.0, compute capability: 7.5
2024-06-14 16:52:24.707014: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2024-06-14 16:52:24.707451: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9566 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:af:00.0, compute capability: 7.5
2024-06-14 16:52:28.480773: I external

24-06-14 16:52:30 - find function duration 9.876848459243774 seconds
match file is 0     /home/liuyichen/deepface/tests/dataset/img10.jpg
1      /home/liuyichen/deepface/tests/dataset/img4.jpg
2      /home/liuyichen/deepface/tests/dataset/img5.jpg
3      /home/liuyichen/deepface/tests/dataset/img2.jpg
4      /home/liuyichen/deepface/tests/dataset/img6.jpg
5     /home/liuyichen/deepface/tests/dataset/img11.jpg
6    /home/liuyichen/deepface/tests/dataset/img11_r...
7      /home/liuyichen/deepface/tests/dataset/img7.jpg
8    /home/liuyichen/deepface/tests/dataset/couple.jpg
9      /home/liuyichen/deepface/tests/dataset/img1.jpg
Name: identity, dtype: object
pointer is 0
pointer is 1
24-06-14 16:52:30 - Searching /home/liuyichen/deepface/tests/dataset/img57.jpg in 70 length datastore
24-06-14 16:52:30 - find function duration 0.3157083988189697 seconds
match file is 0    /home/liuyichen/deepface/tests/dataset/img57.jpg
1    /home/liuyichen/deepface/tests/dataset/img58.jpg
2    /home/liuyic